In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, RocCurveDisplay
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Load your data
data = pd.read_excel("Companies_transpose_edited_2_final.xlsx")
df = pd.DataFrame(data)
df

,Company,Country,Ticker,Year,Revenue,Net Income/Net Profit (Losses),Total Assets,Current Market Cap,Volume,EBITDA,...,Inflation_Rate,Real_GDP_Growth_Rate,ESG_Committee,International_presence,Years in Market,ESG Risk Rating,Moodys Credit Rating,Developed,Market Cap to EBITDA,R&D expense
0,NVIDIA Corporation,USA,NVDA US,2018,9714,3047,11241,147457.98,169569541760,3409,...,2.44,2.875,0.0,1.0,27.0,13.329555,Aa3,1.0,43.255494,1455.0
1,NVIDIA Corporation,USA,NVDA US,2019,11716,4141,13292,97691.5,139011497640,4066,...,1.81,2.326,0.0,1.0,28.0,12.694814,Aa3,1.0,24.026439,1628.0
2,NVIDIA Corporation,USA,NVDA US,2020,10918,2796,17315,153293.76,108382826760,3341,...,1.23,-3.573,0.0,1.0,29.0,12.325062,Aa3,1.0,45.882598,1871.0
3,NVIDIA Corporation,USA,NVDA US,2021,16675,4332,28791,321626.21,123416575600,5775,...,4.70,5.739,1.0,1.0,30.0,11.518750,Aa3,1.0,55.692850,2462.0
4,NVIDIA Corporation,USA,NVDA US,2022,26914,9752,44187,571000,94583088490,11383,...,8.00,1.827,1.0,1.0,31.0,12.125000,Aa3,1.0,50.162523,3076.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
637,"Cloudera, Inc.",USA,CLDR UN,2019,479.941,-192.649,2196.643,3584.4766,144131970,-176.336,...,1.80,2.300,0.0,1.0,11.0,29.000000,B3,1.0,-20.327537,250.0
638,"Cloudera, Inc.",USA,CLDR UN,2020,794.191,-336.582,2299.79,2954.7889,299390910,-217.688,...,1.20,-3.500,0.0,1.0,12.0,28.000000,B3,1.0,-13.573504,300.0
639,"Cloudera, Inc.",USA,CLDR UN,2021,869.258,-162.734,2508.601,4775.765,275932932,-35.855,...,4.70,5.900,0.0,1.0,13.0,27.000000,B3,1.0,-133.196625,350.0
640,"Cloudera, Inc.",USA,CLDR UN,2022,869.258,0,1939.21125,3489.7292,275932932,-197.98725,...,8.00,2.100,0.0,1.0,14.0,26.000000,B3,1.0,-17.626030,400.0


In [3]:
# --- Keep your original data info code for reference ---
print(df.info())
print(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 642 entries, 0 to 641
Data columns (total 48 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   Company                                642 non-null    object 
 1   Country                                642 non-null    object 
 2   Ticker                                 642 non-null    object 
 3   Year                                   642 non-null    int64  
 4   Revenue                                639 non-null    object 
 5   Net Income/Net Profit (Losses)         639 non-null    object 
 6   Total Assets                           639 non-null    object 
 7   Current Market Cap                     638 non-null    object 
 8   Volume                                 639 non-null    object 
 9   EBITDA                                 639 non-null    object 
 10  Net Debt                               639 non-null    object 
 11  Total 

In [4]:
print(df.describe(include='all'))

                   Company Country   Ticker         Year Revenue  \
count                  642     642      642   642.000000     639   
unique                 107      24      112          NaN     628   
top     NVIDIA Corporation     USA  NVDA US          NaN       —   
freq                     6     162        6          NaN       3   
mean                   NaN     NaN      NaN  2020.500000     NaN   
std                    NaN     NaN      NaN     1.709157     NaN   
min                    NaN     NaN      NaN  2018.000000     NaN   
25%                    NaN     NaN      NaN  2019.000000     NaN   
50%                    NaN     NaN      NaN  2020.500000     NaN   
75%                    NaN     NaN      NaN  2022.000000     NaN   
max                    NaN     NaN      NaN  2023.000000     NaN   

        Net Income/Net Profit (Losses) Total Assets Current Market Cap Volume  \
count                            639.0          639                638    639   
unique               

In [5]:
# --- 2. Definir tipos de datos por columna ---
columns_types = {
    # Strings
    'Company': str,
    'Country': str,
    'Ticker': str,
    'Moody\'s Credit Rating': str,
    
    # Integers
    'Year': 'Int64',
    'Years in Market': 'Int64',
    'Volume': 'Int64',

    # Booleans
    'ESG_Committee': 'boolean',
    'Developed': 'boolean',
    'International_presence': 'boolean',
    
    # Floats
    'Revenue': float,
    'Net Income/Net Profit (Losses)': float,
    'Total Assets': float,
    'Current Market Cap': float,
    'EBITDA': float,
    'Net Debt': float,
    'Total Equity': float,
    'ROIC': float,
    'EBIT': float,
    'Net interest': float,
    'CAPEX': float,
    'Total Liabilities': float,
    'Total Debt': float,
    'Quick ratio': float,
    'Cash and equiv': float,
    'Mkt securities': float,
    'Asset Turnover': float,
    'Net marging': float,
    'Debt/Equity': float,
    'Log(Total Assets)': float,
    'Ln(revenue)': float,
    'ROE': float,
    '(EBIT-CapEx)/Interest Expense': float,
    'ROA': float,
    'Debt to EBITDA': float,
    'Debt Ratio': float,
    'Debt/Book Capitalization': float,
    'Debt/Equity Ratio': float,
    'Debt/Total Capitalization': float,
    '(Cash + Marketable Securities) / Debt': float,
    'Market Cap to Revenue ratio': float,
    '(P/B) ratio': float,
    'GDP_per_Capita': float,
    'Inflation_Rate': float,
    'Real_GDP_Growth_Rate': float,
    'ESG Risk Rating': float,
    'Market Cap to EBITDA': float,
    'R&D expense': float,
}

# --- 3. Reemplazar coma decimal si aplica ---
for col in df.columns:
    if df[col].dtype == object:
        df[col] = df[col].astype(str).str.replace(',', '.', regex=False)

# --- 4. Convertir columnas a sus tipos correspondientes ---
for col, dtype in columns_types.items():
    try:
        if dtype == 'boolean':
            df[col] = df[col].map({'1': True, '0': False, 1: True, 0: False}).astype('boolean')
        else:
            df[col] = pd.to_numeric(df[col], errors='coerce') if dtype in [float, 'Int64'] else df[col].astype(dtype)
    except Exception as e:
        print(f"Error converting column {col}: {e}")

# --- 5. Verifica los resultados ---
print(df.dtypes)
print(df.head())

Error converting column Moody's Credit Rating: "Moody's Credit Rating"
Company                                   object
Country                                   object
Ticker                                    object
Year                                       int64
Revenue                                  float64
Net Income/Net Profit (Losses)           float64
Total Assets                             float64
Current Market Cap                       float64
Volume                                   float64
EBITDA                                   float64
Net Debt                                 float64
Total Equity                             float64
ROIC                                     float64
EBIT                                     float64
Net interest                             float64
CAPEX                                    float64
Total Liabilities                        float64
Total Debt                               float64
Quick ratio                              float6

In [6]:
print(df.describe(include='all'))

                   Company Country   Ticker         Year        Revenue  \
count                  642     642      642   642.000000     636.000000   
unique                 107      24      112          NaN            NaN   
top     NVIDIA Corporation     USA  NVDA US          NaN            NaN   
freq                     6     162        6          NaN            NaN   
mean                   NaN     NaN      NaN  2020.500000   27735.984613   
std                    NaN     NaN      NaN     1.709157   63733.754464   
min                    NaN     NaN      NaN  2018.000000   -1914.481100   
25%                    NaN     NaN      NaN  2019.000000    1421.418750   
50%                    NaN     NaN      NaN  2020.500000    5957.918400   
75%                    NaN     NaN      NaN  2022.000000   22617.750000   
max                    NaN     NaN      NaN  2023.000000  574785.000000   

        Net Income/Net Profit (Losses)   Total Assets  Current Market Cap  \
count                 